In [2]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
from networkx import shortest_path
from networkx import set_edge_attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Graph

In [1]:
min_weight = 5
max_weight = 15
n_nodes = 7 * 10 ** 4
n_edges = 2 * 10 ** 6
# n_nodes, n_edges = 50000, 1000000 # <|-- takes 17.5 s to compute the maximum flow in a graph
source, destination = 0, 1 # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [3]:
def modify_weights_graph (Graph, ctr_lists, new_weights) :
    """ utility function for setting weights (dual problem of shortest path) """
    set_edge_attributes(Graph, 0, name="weight")

    set_edge_attributes(
        Graph,
        values = dict({edge : dual for edge, dual in zip (ctr_lists, new_weights)}),
        name = 'weight'
    )

In [4]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)
Graph_dual = deepcopy(Graph)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [5]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

C:\Users\val\AppData\Local\Temp\ipykernel_2548\2680887831.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj_matrix = adjacency_matrix(Graph)


(70000, 70000)

In [6]:
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')
# total flow, flow first node
flow, flow_dict[0]

(259.80696737823627,
 {21204: 7.338335110364076,
  50678: 8.96299054981208,
  56915: 8.37935837340329,
  26334: 10.720195322041421,
  56284: 13.04473558402868,
  4578: 14.74711428214798,
  6605: 12.554742019878455,
  24890: 14.187743448360347,
  8318: 7.269304828595172,
  9374: 14.78112626018133,
  49866: 9.987085249499334,
  24163: 8.44645652701417,
  14389: 8.318402824703455,
  38817: 6.32005015137176,
  51022: 9.203809446375338,
  16878: 10.666467199822785,
  49001: 9.84380522214076,
  17130: 9.65759125788502,
  32087: 8.169279008376495,
  15211: 13.726548897751593,
  56006: 13.548004410872762,
  37475: 6.560585444485313,
  26767: 13.045569639856232,
  50519: 13.749037264014701,
  4056: 6.578629055253767})

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [7]:
import pulp as pl

In [8]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

'cplex.exe'

In [9]:
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=10)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [10]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=50))

50

## Master

In [14]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)
lpProb += 0 # initiate the objective function to 0 (necessary line of code)

In [15]:
# useful to know the final paths at the end and calculate the shortest path for dual problem
path2idx = {}
ctr_lists = []
path_count = 0

In [16]:
def add_paths (lpProb, paths, path_count, path2idx):
    """ Add paths in lpPoblem (new variables Xp and new constraints or terms in constraints Ce) """

    # pour chaque path ajouter une variable et ses coef. dans les contraintes
    for path in paths :

        # add a var path
        prov = pl.LpVariable("X"+str(path_count), 0, max_weight, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)
        
        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path)-1) :
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
            else:
                lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]
                ctr_lists.append((path[index], path[index + 1]))
        
        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

In [17]:
path_count, path2idx = add_paths (lpProb, get_n_paths (Graph, source, destination, max_iter = 1), path_count, path2idx)

In [18]:
# solver = pl.CPLEX_PY()
result = lpProb.solve()
# result = lpProb.solve(pl.GUROBI_CMD(options=[("MIPgap", 0.9)]))

## Dual variables

In [19]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

## Iterative procedure

In [20]:
go_on = True

while go_on :
    # search of path q (dual problem)
    duals = get_duals(lpProb)
    modify_weights_graph (Graph_dual, ctr_lists, duals)
    path_q = shortest_path(Graph_dual, source, destination, weight="weight")

    # compute reduced cost for dual
    reduced_cost = 1
    for e in range(len(path_q)-1) :
        try :
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError : # <|-- constraint does not exist (so dual == 0)
            pass

    # stop procedure ?
    if reduced_cost > 0 : # add path q in lp Problem
        path_count, path2idx = add_paths(lpProb, [path_q], path_count, path2idx)
        lpProb.solve()
    else :
        go_on = False

In [21]:
flow = 0
for var in lpProb.variables():
    flow += var.value()
print(flow)

259.8069673299999


In [22]:
paths = {path2idx[int(str(var)[1:])] : var.value() for var in lpProb.variables() if var.value() > 0}
paths

{'[0, 21204, 5585, 13874, 13610, 69681, 52877, 14109, 14262, 40703, 1]': 1.8656445,
 '[0, 49866, 13224, 57376, 1]': 6.3299619,
 '[0, 21204, 37326, 33057, 29307, 1]': 5.4726906,
 '[0, 50678, 8909, 8977, 57069, 1]': 2.5134126,
 '[0, 56915, 50394, 40636, 34472, 1]': 5.1674056,
 '[0, 56915, 69429, 68380, 33333, 1]': 3.2119528,
 '[0, 26334, 7389, 20398, 51402, 1]': 6.4350903,
 '[0, 26334, 68699, 48486, 16004, 1]': 4.285105,
 '[0, 56284, 64287, 20738, 16004, 1]': 6.0485494,
 '[0, 4578, 44252, 9207, 38849, 1]': 0.92040728,
 '[0, 38817, 61820, 54506, 1]': 6.3200502,
 '[0, 4578, 27237, 16235, 56616, 1]': 6.569066,
 '[0, 4578, 9927, 42676, 33333, 1]': 7.257641,
 '[0, 6605, 33460, 100, 38849, 1]': 8.520393,
 '[0, 6605, 1225, 64591, 9446, 1]': 4.034349,
 '[0, 24890, 64592, 11660, 28250, 1]': 6.5978205,
 '[0, 24890, 12669, 58200, 51402, 1]': 5.0729107,
 '[0, 24890, 42183, 14061, 37567, 22376, 1]': 2.5170122,
 '[0, 8318, 53587, 63060, 38884, 1]': 1.5999712,
 '[0, 9374, 49607, 46764, 8129, 1]': 3.144